In [1]:
import pandas as pd
import numpy as np

# See what products didn't sell in 2021

inv = pd.read_csv('InventoryList_2022-02-04.csv',header=0,dtype=str)

inv = inv[['ProductCode','Name','Category','Barcode','PriceTier1','Status']].copy()
inv = inv[inv.Status == 'ACTIVE']
inv.head()

sold = pd.read_excel('2021details.xlsx',engine='openpyxl',header=0,skiprows=5, parse_dates=False)
custs = ['Customer Square ' + str(x) for x in range(1,4)] #we have 3 stores active, we won't count the Warehouse
retail_names = ['Airport','Pleasant Valley','Oak Hill'] # make sure this matches the above list length - no known automation possible

sold = sold[sold.Customer.isin(custs) & sold.Product.notnull()].reset_index(drop=True) #remove wholesale and weird null anomoly entries
sold = sold.drop(['COGS','Profit'], axis=1, inplace=False)#not trusted data columns or useful, we'll ignore them for now

quick = sold.groupby(['Product','SKU','Customer']).Quantity.sum().reset_index()
#quick.info()

dead_map = []

for i, row in inv[inv.Status == 'ACTIVE'].iterrows():
    prod = row.Name
    for cust in custs:
        if(quick[(quick.Product == prod) & (quick.Customer == cust)].empty):
            dead_map.append([prod,cust,row.Category,row.PriceTier1])
            
#print(dead_map)

output = pd.DataFrame(dead_map,columns=['Product','Customer','Category','Retail'])
output

output.to_csv('NeverSoldFULL.csv',index=False)

# Do it again but more smarter, eventually run a longer report

In [13]:
df = pd.read_csv('AvailabilityReport_2022-03-24.csv',header=0,dtype=str)

df = df[['Category','SKU','ProductName','Location','Status','OnHand']].copy()
df.OnHand = df.OnHand.astype(float)
df.OnHand = df.OnHand.astype(int)
df.rename(columns={'ProductName':'Product'},inplace=True)
df = df[df.Location != 'Warehouse']

retail_names = ['Airport','Pleasant Valley','Oak Hill'] # no way known to build map otherwise
custs = ['Customer Square ' + str(x) for x in range(1,len(retail_names)+1)] #we have 3 stores active, we won't count the Warehouse
store_map = dict(zip(retail_names,custs))
df['Customer'] = df.apply(lambda row: store_map[row.Location],axis=1)

#df[(df.Status == 'DEPRECATED') & (df.Location == 'Customer Square 3')].head()#good, its empty
df.head()

,Category,SKU,Product,Location,Status,OnHand,Customer
0,Non Alcoholic Beverage,777823111113,1 Water 355ml,Airport,ACTIVE,123,Customer Square 1
1,Non Alcoholic Beverage,777823111113,1 Water 355ml,Oak Hill,ACTIVE,1,Customer Square 3
2,Non Alcoholic Beverage,777823111113,1 Water 355ml,Pleasant Valley,ACTIVE,5,Customer Square 2
3,Bar Tools,1234500000000,12/24 Mixing Tin Sets,Airport,ACTIVE,0,Customer Square 1
4,Bar Tools,1234500000000,12/24 Mixing Tin Sets,Pleasant Valley,ACTIVE,0,Customer Square 2


In [14]:
#df[(df.Product.str.contains('Del Maguey')) & (df.Location == 'Oak Hill')]

In [16]:
sale = pd.read_excel('last365posthospital.xlsx',engine='openpyxl',header=0,skiprows=5, parse_dates=False)#sales by product summary last 365 no totals export
sale = sale.drop(['Unit','Invoice','Profit'],axis=1).reset_index()#COGS too
sale = sale[(sale.Customer.isin(custs)) & (sale.Product.notnull())]
sale['Unit'] = sale.Sale / sale.Quantity
sale.head()

KeyError: "['Unit' 'Invoice'] not found in axis"

In [ ]:
#sale[(sale.Product.str.contains('Del Maguey')) & (sale.Customer == 'Customer Square 3')]

In [ ]:
master = df.merge(right=sale,on=['Product','Customer'],how='outer')
master.drop(['index'],axis=1,inplace=True)
master.Unit = master.apply(lambda row: master[master.Product == row.Product].Unit.max(),axis=1)
master["StockValue"] = master.OnHand * master.Unit
master = master.reindex(columns=['SKU','Product','Customer','Location','Status','Category_x','Category_y','OnHand','Quantity','Sale','Unit','StockValue'])
master.rename(columns={'Category_x':'CategoryDear','Category_y':'CategorySale'},inplace=True)
master.iloc[3:10]# shows a few cases of where NaN rows matter

In [ ]:
master = master[(master.OnHand > 0) | (master.Quantity.notnull()) & (master.SKU.notnull())] #get rid of things we probably just don't support anymore and stupid anomolies
master.iloc[3:10]

In [ ]:
#jared = smart[smart.Customer == 'Customer Square 3'].groupby('Category').Product.count()
dead_cats = master.groupby(['CategoryDear','Customer']).CategorySale.count()
total_cats = master.groupby(['CategoryDear','Customer']).CategoryDear.count()
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#    print(jared)
#print(dead_cats,"\n",total_cats)

In [ ]:
master[(master.OnHand > 2) & (master.Quantity.isnull()) & (master.Location == 'Oak Hill')].to_csv('tempOHout.csv',index=False)

In [ ]:
t1 = dead_cats.to_frame()
t1 = t1.rename(columns={'CategorySale':'Category'})

t2 = total_cats.to_frame()
t2 = t2.rename(columns={'CategoryDear':'Category'})

cats = t1.merge(right=t2,on=['CategoryDear','Customer'],suffixes=('_Sold', '_Total'))
cats = cats.reset_index()
cats.head()

In [ ]:
#now we create a column of savings for if we dropped all the dead weight per row to show savings
cats["Savings"] = cats.apply(lambda row: master[(master.CategoryDear == row.CategoryDear) & (master.Customer == row.Customer) & (master.Quantity.isnull())].StockValue.sum(),axis=1)
cats.to_csv('DeadCats.csv',index=False)

In [ ]:
cats.groupby('Customer').Savings.sum()
#cats.Savings.sum()/1.35